# Model Based Collaborative Filtering with fastai

Source: [Fastai chapter 8](https://github.com/fastai/fastbook/blob/master/08_collab.ipynb) and [Collab learner docs](https://docs.fast.ai/collab.html)

In [ ]:
from fastai.collab import *
from fastai.tabular.all import *
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
batch_size = 64
num_latent_factors = 50
rating_range=(0, 5.5)
learning_rate = 5e-3
weight_decay = 0.1
num_epoch = 5
layers = [100,50]

In [ ]:
path = untar_data(URLs.ML_100k)

█ |----------------------------------------| 0.00% [0/4924029 00:00<?] |----------------------------------------| 0.17% [8192/4924029 00:00<01:06] |----------------------------------------| 0.50% [24576/4924029 00:00<00:22] |----------------------------------------| 1.50% [73728/4924029 00:00<00:14] |█---------------------------------------| 2.99% [147456/4924029 00:00<00:10] |█---------------------------------------| 4.82% [237568/4924029 00:00<00:08] |██--------------------------------------| 6.99% [344064/4924029 00:00<00:05] |████------------------------------------| 10.15% [499712/4924029 00:00<00:04] |█████-----------------------------------| 13.81% [679936/4924029 00:00<00:03] |███████---------------------------------| 18.80% [925696/4924029 00:00<00:02] |█████████-------------------------------| 24.46% [1204224/4924029 00:00<00:02] |████████████----------------------------| 31.78% [1564672/4924029 00:00<00:01] |███████████████-------------------------| 39.93% [1966080/4924029 00:00<00:01] |███████████████████---------------------| 49.91% [2457600/4924029 00:00<00:00] |████████████████████████----------------| 62.39% [3072000/4924029 00:00<00:00] |██████████████████████████████----------| 76.03% [3743744/4924029 00:00<00:00] |█████████████████████████████████████---| 92.50% [4554752/4924029 00:00<00:00] |████████████████████████████████████████| 100.15% [4931584/4924029 00:00<00:00]

## Import data

In [ ]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

Out[4]:

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

Out[5]:

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
ratings = ratings.merge(movies)
ratings.head()

Out[6]:

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


## Split data

In [ ]:
# todo: for each user keep latest 3 movie reviews as test

In [ ]:
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

## Create Dataloader

In [ ]:
dls = CollabDataLoaders.from_df(train_df, item_name='title', bs=batch_size)

## Fit learner

### Probabilistic matrix factorization

In [ ]:
pmf_learner = collab_learner(dls, n_factors=num_latent_factors, y_range=rating_range)

In [ ]:
pmf_learner.fit_one_cycle(num_epoch, learning_rate, wd=weight_decay)

█epoch train_loss valid_loss time 
█Epoch 1/5 : |----------------------------------------| 0.00% [0/1000 00:00<?]Epoch 1/5 : |----------------------------------------| 0.10% [1/1000 00:00<00:51]Epoch 1/5 : |----------------------------------------| 0.20% [2/1000 00:00<00:31 1.5514]Epoch 1/5 : |----------------------------------------| 0.30% [3/1000 00:00<00:26 1.6351]Epoch 1/5 : |----------------------------------------| 0.40% [4/1000 00:00<00:22 1.7061]Epoch 1/5 : |----------------------------------------| 0.50% [5/1000 00:00<00:20 1.6927]Epoch 1/5 : |----------------------------------------| 1.40% [14/1000 00:00<00:14 1.7454]Epoch 1/5 : |█---------------------------------------| 2.70% [27/1000 00:00<00:12 1.7765]Epoch 1/5 : |█---------------------------------------| 4.20% [42/1000 00:00<00:21 1.8531]Epoch 1/5 : |██--------------------------------------| 5.00% [50/1000 00:01<00:19 1.8883]Epoch 1/5 : |██--------------------------------------| 5.90% [59/1000 00:01<00:18 1.8965]Epoch 1/5 : |██--------------------------------------| 6.80% [68/1000 00:01<00:18 1.8954]Epoch 1/5 : |███-------------------------------------| 7.80% [78/1000 00:01<00:17 1.8756]Epoch 1/5 : |███-------------------------------------| 8.80% [88/1000 00:01<00:16 1.8716]Epoch 1/5 : |███-------------------------------------| 9.90% [99/1000 00:01<00:15 1.8804]Epoch 1/5 : |████------------------------------------| 11.00% [110/1000 00:01<00:14 1.8710]Epoch 1/5 : |████------------------------------------| 12.20% [122/1000 00:01<00:14 1.8690]Epoch 1/5 : |█████-----------------------------------| 13.40% [134/1000 00:02<00:13 1.8644]Epoch 1/5 : |█████-----------------------------------| 14.60% [146/1000 00:02<00:13 1.8627]Epoch 1/5 : |██████----------------------------------| 15.90% [159/1000 00:02<00:12 1.8668]Epoch 1/5 : |██████----------------------------------| 17.20% [172/1000 00:02<00:13 1.8630]Epoch 1/5 : |███████---------------------------------| 18.40% [184/1000 00:02<00:12 1.8574]Epoch 1/5 : |███████---------------------------------| 19.60% [196/1000 00:02<00:12 1.8450]Epoch 1/5 : |████████--------------------------------| 20.90% [209/1000 00:03<00:11 1.8412]Epoch 1/5 : |████████--------------------------------| 22.20% [222/1000 00:03<00:11 1.8262]Epoch 1/5 : |█████████-------------------------------| 23.50% [235/1000 00:03<00:11 1.8206]Epoch 1/5 : |█████████-------------------------------| 24.80% [248/1000 00:03<00:10 1.7932]Epoch 1/5 : |██████████------------------------------| 26.10% [261/1000 00:03<00:10 1.7764]Epoch 1/5 : |██████████------------------------------| 27.40% [274/1000 00:04<00:10 1.7887]Epoch 1/5 : |███████████-----------------------------| 28.70% [287/1000 00:04<00:10 1.7812]Epoch 1/5 : |████████████----------------------------| 30.00% [300/1000 00:04<00:10 1.7813]Epoch 1/5 : |████████████----------------------------| 31.30% [313/1000 00:04<00:09 1.7857]Epoch 1/5 : |█████████████---------------------------| 32.60% [326/1000 00:04<00:09 1.7761]Epoch 1/5 : |█████████████---------------------------| 33.90% [339/1000 00:04<00:09 1.7572]Epoch 1/5 : |██████████████--------------------------| 35.30% [353/1000 00:04<00:09 1.7488]Epoch 1/5 : |██████████████--------------------------| 36.70% [367/1000 00:05<00:08 1.7400]Epoch 1/5 : |███████████████-------------------------| 38.10% [381/1000 00:05<00:08 1.7386]Epoch 1/5 : |███████████████-------------------------| 39.50% [395/1000 00:05<00:08 1.7045]Epoch 1/5 : |████████████████------------------------| 40.90% [409/1000 00:05<00:08 1.7025]Epoch 1/5 : |████████████████------------------------| 42.30% [423/1000 00:05<00:07 1.6858]Epoch 1/5 : |█████████████████-----------------------| 43.70% [437/1000 00:05<00:07 1.6572]Epoch 1/5 : |██████████████████----------------------| 45.10% [451/1000 00:06<00:07 1.6435]Epoch 1/5 : |██████████████████----------------------| 46.50% [465/1000 00:06<00:07 1.6223]Epoch 1/5 : |███

In [ ]:
pmf_learner.model

Out[12]: EmbeddingDotBias(
 (u_weight): Embedding(944, 50)
 (i_weight): Embedding(1639, 50)
 (u_bias): Embedding(944, 1)
 (i_bias): Embedding(1639, 1)
)

## Deep learning

In [ ]:
nn_learner = collab_learner(dls, use_nn=True, y_range=rating_range, layers=layers)

In [ ]:
nn_learner.fit_one_cycle(num_epoch, learning_rate, wd=weight_decay)

█epoch train_loss valid_loss time 
█Epoch 1/5 : |----------------------------------------| 0.00% [0/1000 00:00<?]Epoch 1/5 : |----------------------------------------| 0.10% [1/1000 00:00<01:19]Epoch 1/5 : |----------------------------------------| 0.20% [2/1000 00:00<00:49 2.4682]Epoch 1/5 : |----------------------------------------| 0.30% [3/1000 00:00<00:50 2.3428]Epoch 1/5 : |----------------------------------------| 0.40% [4/1000 00:00<00:50 2.1790]Epoch 1/5 : |----------------------------------------| 0.50% [5/1000 00:00<00:44 2.0731]Epoch 1/5 : |----------------------------------------| 0.90% [9/1000 00:00<00:31 2.1303]Epoch 1/5 : |----------------------------------------| 1.50% [15/1000 00:00<00:26 2.0820]Epoch 1/5 : |----------------------------------------| 2.20% [22/1000 00:00<00:23 1.9736]Epoch 1/5 : |█---------------------------------------| 3.00% [30/1000 00:00<00:22 1.9319]Epoch 1/5 : |█---------------------------------------| 3.80% [38/1000 00:00<00:20 1.8918]Epoch 1/5 : |█---------------------------------------| 4.70% [47/1000 00:01<00:20 1.8223]Epoch 1/5 : |██--------------------------------------| 5.60% [56/1000 00:01<00:19 1.7926]Epoch 1/5 : |██--------------------------------------| 6.50% [65/1000 00:01<00:20 1.7403]Epoch 1/5 : |██--------------------------------------| 7.40% [74/1000 00:01<00:19 1.7147]Epoch 1/5 : |███-------------------------------------| 8.30% [83/1000 00:01<00:19 1.6802]Epoch 1/5 : |███-------------------------------------| 9.20% [92/1000 00:01<00:19 1.6194]Epoch 1/5 : |████------------------------------------| 10.10% [101/1000 00:02<00:19 1.6029]Epoch 1/5 : |████------------------------------------| 11.00% [110/1000 00:02<00:18 1.5608]Epoch 1/5 : |████------------------------------------| 11.90% [119/1000 00:02<00:18 1.5416]Epoch 1/5 : |█████-----------------------------------| 12.80% [128/1000 00:02<00:18 1.5005]Epoch 1/5 : |█████-----------------------------------| 13.70% [137/1000 00:02<00:17 1.4675]Epoch 1/5 : |█████-----------------------------------| 14.60% [146/1000 00:02<00:17 1.4560]Epoch 1/5 : |██████----------------------------------| 15.50% [155/1000 00:03<00:17 1.4332]Epoch 1/5 : |██████----------------------------------| 16.40% [164/1000 00:03<00:16 1.4203]Epoch 1/5 : |██████----------------------------------| 17.30% [173/1000 00:03<00:16 1.3933]Epoch 1/5 : |███████---------------------------------| 18.20% [182/1000 00:03<00:16 1.3814]Epoch 1/5 : |███████---------------------------------| 19.20% [192/1000 00:03<00:15 1.3527]Epoch 1/5 : |████████--------------------------------| 20.20% [202/1000 00:04<00:15 1.3175]Epoch 1/5 : |████████--------------------------------| 21.20% [212/1000 00:04<00:15 1.3053]Epoch 1/5 : |████████--------------------------------| 22.20% [222/1000 00:04<00:15 1.2904]Epoch 1/5 : |█████████-------------------------------| 23.20% [232/1000 00:04<00:14 1.2755]Epoch 1/5 : |█████████-------------------------------| 24.20% [242/1000 00:04<00:14 1.2484]Epoch 1/5 : |██████████------------------------------| 25.20% [252/1000 00:04<00:14 1.2380]Epoch 1/5 : |██████████------------------------------| 26.20% [262/1000 00:05<00:14 1.2154]Epoch 1/5 : |██████████------------------------------| 27.20% [272/1000 00:05<00:14 1.1851]Epoch 1/5 : |███████████-----------------------------| 28.20% [282/1000 00:05<00:13 1.1584]Epoch 1/5 : |███████████-----------------------------| 29.20% [292/1000 00:05<00:13 1.1500]Epoch 1/5 : |████████████----------------------------| 30.20% [302/1000 00:05<00:13 1.1505]Epoch 1/5 : |████████████----------------------------| 31.20% [312/1000 00:05<00:13 1.1165]Epoch 1/5 : |████████████----------------------------| 32.20% [322/1000 00:06<00:12 1.1256]Epoch 1/5 : |█████████████---------------------------| 33.20% [332/1000 00:06<00:12 1.1203]Epoch 1/5 : |█████████████---------------------------| 34.20% [342/1000 00:06<00:12 1.1171]Epoch 1/5 : |████████

In [ ]:
nn_learner.model

Out[15]: EmbeddingNN(
 (embeds): ModuleList(
 (0): Embedding(944, 74)
 (1): Embedding(1639, 101)
 )
 (emb_drop): Dropout(p=0.0, inplace=False)
 (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 (layers): Sequential(
 (0): LinBnDrop(
 (0): Linear(in_features=175, out_features=100, bias=False)
 (1): ReLU(inplace=True)
 (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 (1): LinBnDrop(
 (0): Linear(in_features=100, out_features=50, bias=False)
 (1): ReLU(inplace=True)
 (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )
 (2): LinBnDrop(
 (0): Linear(in_features=50, out_features=1, bias=True)
 )
 (3): fastai.layers.SigmoidRange(low=0, high=5.5)
 )
)

## Baseline

Median rating.

In [ ]:
median_rating = train_df.rating.median()

## Evaluation of recommendations

For ratings: mean squared error

In [ ]:
base_y_pred = np.array([median_rating] * len(test_df))

In [ ]:
pmf_y_pred, y_true = pmf_learner.get_preds(dl=pmf_learner.dls.test_dl(test_df))

█ |----------------------------------------| 0.00% [0/313 00:00<?] |----------------------------------------| 0.32% [1/313 00:00<00:01] |----------------------------------------| 0.64% [2/313 00:00<00:01 0.4503] |----------------------------------------| 0.96% [3/313 00:00<00:01 0.4503] |----------------------------------------| 1.28% [4/313 00:00<00:01 0.4503] |----------------------------------------| 1.60% [5/313 00:00<00:01 0.4503] |█████-----------------------------------| 13.74% [43/313 00:00<00:01 0.4503] |██████████------------------------------| 26.20% [82/313 00:00<00:01 0.4503] |███████████████-------------------------| 38.98% [122/313 00:00<00:00 0.4503] |████████████████████--------------------| 51.76% [162/313 00:00<00:00 0.4503] |█████████████████████████---------------| 64.54% [202/313 00:01<00:00 0.4503] |██████████████████████████████----------| 77.32% [242/313 00:01<00:00 0.4503] |████████████████████████████████████----| 90.10% [282/313 00:01<00:00 0.4503] |████████████████████████████████████████| 100.00% [313/313 00:01<00:00 0.4503]

In [ ]:
nn_y_pred, y_true = nn_learner.get_preds(dl=nn_learner.dls.test_dl(test_df))

█ |----------------------------------------| 0.00% [0/313 00:00<?] |----------------------------------------| 0.32% [1/313 00:00<00:02] |----------------------------------------| 0.64% [2/313 00:00<00:01 0.7797] |----------------------------------------| 0.96% [3/313 00:00<00:01 0.7797] |----------------------------------------| 1.28% [4/313 00:00<00:01 0.7797] |----------------------------------------| 1.60% [5/313 00:00<00:01 0.7797] |████------------------------------------| 12.14% [38/313 00:00<00:01 0.7797] |████████--------------------------------| 21.09% [66/313 00:00<00:01 0.7797] |████████████----------------------------| 30.35% [95/313 00:00<00:01 0.7797] |████████████████------------------------| 40.26% [126/313 00:00<00:01 0.7797] |████████████████████--------------------| 50.16% [157/313 00:00<00:00 0.7797] |████████████████████████----------------| 60.38% [189/313 00:01<00:00 0.7797] |████████████████████████████------------| 70.29% [220/313 00:01<00:00 0.7797] |████████████████████████████████--------| 80.19% [251/313 00:01<00:00 0.7797] |████████████████████████████████████----| 90.10% [282/313 00:01<00:00 0.7797] |████████████████████████████████████████| 100.00% [313/313 00:01<00:00 0.7797]

In [ ]:
base_mse = mean_squared_error(base_y_pred, y_true)
pmf_mse = mean_squared_error(pmf_y_pred, y_true)
nn_mse = mean_squared_error(nn_y_pred, y_true)

In [ ]:
print(f'Baseline MSE {base_mse}, PMF MSE: {pmf_mse}, NN MSE: {nn_mse}')

Baseline MSE 1.50905, PMF MSE: 0.8529424447300584, NN MSE: 0.8825892347138049